In [1]:
import sys
sys.path.append('../')
sys.dont_write_bytecode = True
import os
import numpy as np
import scipy.ndimage
from matplotlib.pyplot import *
%matplotlib inline
import numba
sys.path.append('/Users/Michael_Saha/Repositories/')
sys.path.append('./../../../')
from fire_sim import *

In [3]:
# Break into classes:
#
#    Landscape -> Environmental variables and fire history
#    Town -> Container for agents to loop over
#    
#    Calibration -> Contains a Landscape and manages the training/evaluation of 
#        fire behaviour against historical fire data.
#
#    ABM -> Container for parameterized Landscape and Towns of peoples.
#        Town (and home) behaviours depend on the Landscape and Town behaviours.

In [5]:
euclidean_distance(0,0,1,1)

1.4142135623730951

In [10]:
@numba.jit(nopython=True)
def foo(x, y):
    return x + y

y = np.array([1,2,3,4,5,6])

foo(6,y)
foo.inspect_types()

foo (int64, array(int64, 1d, C))
--------------------------------------------------------------------------------
# File: <ipython-input-10-8e7ea949e6e4>
# --- LINE 1 --- 

@numba.jit(nopython=True)

# --- LINE 2 --- 

def foo(x, y):

    # --- LINE 3 --- 
    # label 0
    #   x = arg(0, name=x)  :: int64
    #   y = arg(1, name=y)  :: array(int64, 1d, C)
    #   $0.3 = x + y  :: array(int64, 1d, C)
    #   del y
    #   del x
    #   $0.4 = cast(value=$0.3)  :: array(int64, 1d, C)
    #   del $0.3
    #   return $0.4

    return x + y




In [36]:
class Town:
    def __init__(self, homes, initial_ds, betas, model):
        """Initialize a Town with homes."""
        self.homes = homes
        self.ds = initial_ds
        self.betas = betas
        self.model = model


@numba.jit(nopython=True)
def min_distances(A, B, distances):
    """Find the minimum distance of each pixel in A to any pixel in B."""
    assert A[0].size == A[1].size
    assert B[0].size == B[1].size
    assert A[0].size > 0 and B[0].size > 0
    for i in range(A[0].size):
        distances[i] = euclidean_distance(A[0][i], A[1][i], B[0][0], B[1][0])
        for j in range(B[0].size):
            d = euclidean_distance(A[0][i], A[1][i], B[0][j], B[1][j])
            if d < distances[i]:
                distances[i] = d

class ABM:
    """Coupled Towns and Landscapes"""
    def __init__(towns, landscape):
        assert towns in landscapes
        
    def nearest_fire(self):
        for town in self.towns:
            for fire in zip(self.landscape.past_fires, self.landscape.past_fire_map)
                distance_to_last_fire(town.homes, fire, distances)
    
    def update_ds(self):
        """Update defensible space on the landscape using the thinning algorithm."""
        

In [46]:
A = np.arange(1000), np.arange(1000)
B = np.arange(100,200), np.arange(100,200)

distance = np.zeros(A[0].size)
%timeit d = min_distances(A, B, distance)

1000 loops, best of 3: 948 µs per loop


In [ ]:
# tuple of numpy arrays
# 